# This Note showcases the RAG usecase
- Uses Langchain
- FAISS vector Store
- Demonstrates, how to split the documents into multiple chuncks
- Demonstrates, how to query the embedings from the vector store
- Demostrates, calling BAM models for the query

In [1]:
!pip install sentence_transformers
!pip install InstructorEmbedding
!pip install langchain
!pip install ibm-generative-ai
!conda install -c conda-forge faiss -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 1.7 MB/s eta 0:00:00


  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.0.1
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.8.2
    Uninstalling pydantic-1.8.2:
      Successfully uninstalled pydantic-1.8.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.4.0.dev0 requires torch>=1.13.0, but you have torch 1.12.1 which is incompatible.
gradio 3.23.0 requires huggingface-hub>=0.13.0, but you have huggingface-hub 0.10.1 which is incompatible.
allennlp 2.10.1 requi

libfaiss-1.7.4       | 1.3 MB    | 4                                     |   1% 
libblas-3.9.0        | 13 KB     | ##################################### | 100% 




libcxx-16.0.5        | 1.1 MB    | 5                                     |   1% 


openssl-1.1.1u       | 1.7 MB    | #########7                            |  26% 

libfaiss-1.7.4       | 1.3 MB    | ##########                            |  27% 



llvm-openmp-16.0.5   | 289 KB    | ##################################### | 100% 



llvm-openmp-16.0.5   | 289 KB    | ##################################### | 100% 




libcxx-16.0.5        | 1.1 MB    | ###########6                          |  32% 

libfaiss-1.7.4       | 1.3 MB    | #############################2        |  79% 


openssl-1.1.1u       | 1.7 MB    | ################################8     |  89% 




libfaiss-avx2-1.7.4  | 1.4 MB    | ################9                     |  46% 





liblapack-3.9.0      | 13 KB     | ##################################### | 100% 

In [2]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

#BAM
from genai.extensions.langchain import LangChainInterface
from genai.schemas import ModelType, GenerateParams
from genai.model import Credentials

import getpass

import os

## Pass Watsonx.ai API Key

In [3]:
api_key="your-token-key"
my_api_endpoint="https://fmaas-dev-api.bx.cloud9.ibm.com/v1/"

## Global settings

- chunksize: size of chunks documents need to be splited
- chunk_overlap: overlap of the chunks


In [4]:
chunk_size = 2000
chunk_overlap = 100

## Loading the pdf, file using the PyPDFLoader

In [9]:
cd /Users/sahil/Downloads/data

/Users/sahil/Downloads/data


In [13]:
loader = PyPDFLoader("range_data.pdf")
data = loader.load()

## Total number of documents (pages) in the pdf

In [14]:
#every page in pdf is counted as unique document
print (f'You have {len(data)} document(s) in your data')

You have 84 document(s) in your data


## Printing the first page of the pdf file

In [15]:
print (f'There are {len(data[0].page_content)} characters in first page')
print(f"content of first page\n : {data[0].page_content}")

There are 109 characters in first page
content of first page
 : imagine the possibilities
Thank you for purchasing this Samsung product.Microwave Oven
user manualME19R7041F*


## Spliting the documents into multiple chuncks on the chunk size mentioned earlier

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size= chunk_size, chunk_overlap=chunk_overlap)
docs = text_splitter.split_documents(data)

In [17]:
print(f'We have total documents after split: {len(docs)}')

We have total documents after split: 191


## Loading Hugging Face Emedings
- When you run the below cell for the first time, it does take some time

In [18]:
embeddings = HuggingFaceInstructEmbeddings(
            model_name="hkunlp/instructor-large",
            model_kwargs={"device": "cpu"}
        )

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/270 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


## Vector Store- FAISS
- We have our documents and embedding ready.
- Here we are storing our embeddings and docs in the vector store

In [19]:
#https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html?highlight=faiss#faiss
db = FAISS.from_documents(docs, embeddings)

## Lets test our embeddings
- We are passing the query, and looking for the closest 3 embedings.
- printing out the closest 3 embedings for the query from the documents or pdf file

In [20]:
query = "How to cook eggs"
docs = db.similarity_search(query, k=3)
print(len(docs))
print(docs[0].page_content)
print("----")
print(docs[1].page_content)
print("----")
print(docs[2].page_content)

3
English - 3IMPORTANT SAFETY 
INSTRUCTIONS
When using any electrical appliance, basic safety 
precautions should be followed, including the following:
WARNING WARNING
To reduce risk of burns, electric shock, fire, personal 
injury or exposure to excessive microwave energy:
1. Read all safety instructions before using the 
appliance.
2. Read and follow the specific “Precautions to avoid 
possible exposure to excessive microwave energy” on 
this page.
3. This appliance must be grounded. Connect only to 
properly grounded outlets. See Important “Grounding 
instructions” on page 4 of this manual.
4. Install or locate this appliance only in accordance with 
the provided installation instructions.
5. Some products such as whole eggs and sealed 
containers (for example, sealed glass jars), can 
explode if heated rapidly. Never heat them in a 
microwave oven.
6. Use this appliance only for its intended use as 
described in the manual. Do not put corrosive 
chemicals or vapors in or on this ap

## Creating LLM model
- Here we are using LangChainInterface to create out BAM model

In [21]:
model_llm = LangChainInterface(
        model=ModelType.FLAN_T5_11B,
        credentials=Credentials(api_key, api_endpoint=my_api_endpoint),
        params=GenerateParams(
            decoding_method="greedy",
            max_new_tokens=300,
            min_new_tokens=15,
            repetition_penalty=2,
        ).dict()
    )

## Loading lang chain qa
- creating a chain to get QA from our BAM modles.
- Here we are passing chain_type as stuff, which means we are passing all the embeddings fromt the query

In [22]:
chain = load_qa_chain(model_llm, chain_type="stuff")

## Let' get the embedings for the query

In [23]:
query = "How to cook eggs"
doc = db.similarity_search(query, k=3)
print(len(doc))

3


## Finally it's time for us to call our BAM model
- here we are passing all embedding and the query to the BAM models

In [24]:
chain.run(input_documents=docs, question=query)

'Cook them until they are just set as they become tough if they are overcooked.'

## End of the notebook